In [55]:
#libraries
import pandas as pd
import numpy as np

In [56]:
#Import the dataset
from google.cloud import storage
Vehicle_CrimeDf = pd.read_excel('gs://phepha-bucket_1/ml_models/dummy data draft.xlsx', 
                              header = None, skiprows=1, names = ['Place', 'Latitude', 'Longitude', 'Crime_ID', 'Province', 'Physical_Address',
                                                     'Car_Type', 'Year', 'Crime_Type', 'Timestamp', 'Time_Of_Day'])
print(Vehicle_CrimeDf)
Vehicle_CrimeDf.head()

                         Place  Latitude  Longitude  Crime_ID Province  \
0   Altron Systems Integration  -26.0501    28.0569         1  Gauteng   
1   Altron Systems Integration  -26.0501    28.0569         2  Gauteng   
2   Altron Systems Integration  -26.0501    28.0569         3  Gauteng   
3   Altron Systems Integration  -26.0501    28.0569         4  Gauteng   
4   Altron Systems Integration  -26.0501    28.0569         5  Gauteng   
..                         ...       ...        ...       ...      ...   
95          Olivedale Randburg  -26.0557    27.9892        96  Gauteng   
96          Nicolway Bryanston  -26.0521    28.0241        97  Gauteng   
97          Nicolway Bryanston  -26.0521    28.0241        98  Gauteng   
98          Nicolway Bryanston  -26.0521    28.0241        99  Gauteng   
99          Nicolway Bryanston  -26.0521    28.0241       100  Gauteng   

   Physical_Address      Car_Type  Year                Crime_Type  \
0   123 Main Street  Toyota Hilux  2021   

,Place,Latitude,Longitude,Crime_ID,Province,Physical_Address,Car_Type,Year,Crime_Type,Timestamp,Time_Of_Day
0,Altron Systems Integration,-26.0501,28.0569,1,Gauteng,123 Main Street,Toyota Hilux,2021,Hijacking,2021-05-10T08:30:00,Morning
1,Altron Systems Integration,-26.0501,28.0569,2,Gauteng,123 Main Street,Ford Ranger,2021,Remote Jamming,2021-06-20T14:45:00,Afternoon
2,Altron Systems Integration,-26.0501,28.0569,3,Gauteng,123 Main Street,VW Polo,2021,Smash and Grab,2021-07-15T18:20:00,Evening
3,Altron Systems Integration,-26.0501,28.0569,4,Gauteng,123 Main Street,Toyota Hilux,2022,Vehicle Vandalism,2022-02-05T10:10:00,Morning
4,Altron Systems Integration,-26.0501,28.0569,5,Gauteng,123 Main Street,Ford Ranger,2022,Organized Truck Lootings,2022-03-15T13:55:00,Afternoon


In [57]:
#Checking data types of the dataset's variables
print(Vehicle_CrimeDf.dtypes)

Place                object
Latitude            float64
Longitude           float64
Crime_ID              int64
Province             object
Physical_Address     object
Car_Type             object
Year                  int64
Crime_Type           object
Timestamp            object
Time_Of_Day          object
dtype: object


In [58]:
#No need to change the variable's data types since they are read correctly
#No need to check for duplicated columns and handle them
#No need to check for duplicated rows and handle them
#No need to check for missing values and them them

In [59]:
#Drop predictor variables that will not be effective at predicting the response variable
Dropped_Predictor_Variables = ['Crime_ID', 'Province', 'Physical_Address']
DropVehicle_CrimeDf = Vehicle_CrimeDf.drop(Dropped_Predictor_Variables, axis=1)
print(DropVehicle_CrimeDf)

                         Place  Latitude  Longitude      Car_Type  Year  \
0   Altron Systems Integration  -26.0501    28.0569  Toyota Hilux  2021   
1   Altron Systems Integration  -26.0501    28.0569   Ford Ranger  2021   
2   Altron Systems Integration  -26.0501    28.0569       VW Polo  2021   
3   Altron Systems Integration  -26.0501    28.0569  Toyota Hilux  2022   
4   Altron Systems Integration  -26.0501    28.0569   Ford Ranger  2022   
..                         ...       ...        ...           ...   ...   
95          Olivedale Randburg  -26.0557    27.9892   Ford Ranger  2023   
96          Nicolway Bryanston  -26.0521    28.0241  Toyota Hilux  2021   
97          Nicolway Bryanston  -26.0521    28.0241   Ford Ranger  2021   
98          Nicolway Bryanston  -26.0521    28.0241       VW Polo  2021   
99          Nicolway Bryanston  -26.0521    28.0241  Toyota Hilux  2022   

                  Crime_Type            Timestamp Time_Of_Day  
0                  Hijacking  2021-

In [60]:
from sklearn.preprocessing import LabelEncoder
#Encoding categorical variables using "Label Encoding"
# Perform label encoding on categorical variables
label_encoder = LabelEncoder()
for column in ['Car_Type','Time_Of_Day', 'Place', 'Crime_Type', 'Time_Of_Day']:
    DropVehicle_CrimeDf[column] = label_encoder.fit_transform(DropVehicle_CrimeDf[column])

print(DropVehicle_CrimeDf)

    Place  Latitude  Longitude  Car_Type  Year  Crime_Type  \
0       0  -26.0501    28.0569         1  2021           0   
1       0  -26.0501    28.0569         0  2021           2   
2       0  -26.0501    28.0569         2  2021           3   
3       0  -26.0501    28.0569         1  2022           4   
4       0  -26.0501    28.0569         0  2022           1   
..    ...       ...        ...       ...   ...         ...   
95      3  -26.0557    27.9892         0  2023           3   
96      2  -26.0521    28.0241         1  2021           4   
97      2  -26.0521    28.0241         0  2021           1   
98      2  -26.0521    28.0241         2  2021           0   
99      2  -26.0521    28.0241         1  2022           2   

              Timestamp  Time_Of_Day  
0   2021-05-10T08:30:00            2  
1   2021-06-20T14:45:00            0  
2   2021-07-15T18:20:00            1  
3   2022-02-05T10:10:00            2  
4   2022-03-15T13:55:00            0  
..                  .

In [61]:
# Convert timestamp column to datetime dtype
DropVehicle_CrimeDf['Timestamp'] = pd.to_datetime(DropVehicle_CrimeDf['Timestamp'])

# Extract date from timestamp
DropVehicle_CrimeDf['Date'] = DropVehicle_CrimeDf['Timestamp'].dt.date

# Drop the original timestamp column if it's no longer needed
DropVehicle_CrimeDf.drop(columns=['Timestamp'], inplace=True)

# Show the DataFrame with new columns
print(DropVehicle_CrimeDf)

    Place  Latitude  Longitude  Car_Type  Year  Crime_Type  Time_Of_Day  \
0       0  -26.0501    28.0569         1  2021           0            2   
1       0  -26.0501    28.0569         0  2021           2            0   
2       0  -26.0501    28.0569         2  2021           3            1   
3       0  -26.0501    28.0569         1  2022           4            2   
4       0  -26.0501    28.0569         0  2022           1            0   
..    ...       ...        ...       ...   ...         ...          ...   
95      3  -26.0557    27.9892         0  2023           3            0   
96      2  -26.0521    28.0241         1  2021           4            2   
97      2  -26.0521    28.0241         0  2021           1            0   
98      2  -26.0521    28.0241         2  2021           0            1   
99      2  -26.0521    28.0241         1  2022           2            2   

          Date  
0   2021-05-10  
1   2021-06-20  
2   2021-07-15  
3   2022-02-05  
4   2022-03-15

In [62]:
# Extract day, month, and year
DropVehicle_CrimeDf['Day'] = DropVehicle_CrimeDf['Date'].apply(lambda x: x.day)
DropVehicle_CrimeDf['Month'] = DropVehicle_CrimeDf['Date'].apply(lambda x: x.month)
DropVehicle_CrimeDf['Year'] = DropVehicle_CrimeDf['Date'].apply(lambda x: x.year)

# Show the DataFrame with new columns
print(DropVehicle_CrimeDf)

    Place  Latitude  Longitude  Car_Type  Year  Crime_Type  Time_Of_Day  \
0       0  -26.0501    28.0569         1  2021           0            2   
1       0  -26.0501    28.0569         0  2021           2            0   
2       0  -26.0501    28.0569         2  2021           3            1   
3       0  -26.0501    28.0569         1  2022           4            2   
4       0  -26.0501    28.0569         0  2022           1            0   
..    ...       ...        ...       ...   ...         ...          ...   
95      3  -26.0557    27.9892         0  2023           3            0   
96      2  -26.0521    28.0241         1  2021           4            2   
97      2  -26.0521    28.0241         0  2021           1            0   
98      2  -26.0521    28.0241         2  2021           0            1   
99      2  -26.0521    28.0241         1  2022           2            2   

          Date  Day  Month  
0   2021-05-10   10      5  
1   2021-06-20   20      6  
2   2021-07-

In [63]:
# Drop the 'Date' variable
DropVehicle_CrimeDf.drop(columns=['Date'], inplace=True)

# Show the DataFrame with new columns
print(DropVehicle_CrimeDf)

    Place  Latitude  Longitude  Car_Type  Year  Crime_Type  Time_Of_Day  Day  \
0       0  -26.0501    28.0569         1  2021           0            2   10   
1       0  -26.0501    28.0569         0  2021           2            0   20   
2       0  -26.0501    28.0569         2  2021           3            1   15   
3       0  -26.0501    28.0569         1  2022           4            2    5   
4       0  -26.0501    28.0569         0  2022           1            0   15   
..    ...       ...        ...       ...   ...         ...          ...  ...   
95      3  -26.0557    27.9892         0  2023           3            0   25   
96      2  -26.0521    28.0241         1  2021           4            2   18   
97      2  -26.0521    28.0241         0  2021           1            0   30   
98      2  -26.0521    28.0241         2  2021           0            1    5   
99      2  -26.0521    28.0241         1  2022           2            2    8   

    Month  
0       5  
1       6  
2  

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Split the data into features and target variable
X = DropVehicle_CrimeDf[['Place','Car_Type', 'Time_Of_Day', 'Day', 'Month', 'Year']]
y = DropVehicle_CrimeDf[['Latitude', 'Longitude']]

# Perform data normalization
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split the data into training and test sets and applying a stratified sampling method
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=2024, stratify = y)

# Print the shapes of the training and test sets to verify the split
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (70, 6)
X_test shape: (30, 6)
y_train shape: (70, 2)
y_test shape: (30, 2)


In [65]:
import numpy as np

# Calculate the proportions of each class in the original dataset for latitude
original_latitude_proportions = DropVehicle_CrimeDf['Latitude'].value_counts(normalize=True)

# Calculate the proportions of each class in the training set for latitude
train_latitude_proportions = np.unique(y_train['Latitude'], return_counts=True)[1] / len(y_train)

# Calculate the proportions of each class in the test set for latitude
test_latitude_proportions = np.unique(y_test['Latitude'], return_counts=True)[1] / len(y_test)

# Calculate the proportions of each class in the original dataset for longitude
original_longitude_proportions = DropVehicle_CrimeDf['Longitude'].value_counts(normalize=True)

# Calculate the proportions of each class in the training set for longitude
train_longitude_proportions = np.unique(y_train['Longitude'], return_counts=True)[1] / len(y_train)

# Calculate the proportions of each class in the test set for longitude
test_longitude_proportions = np.unique(y_test['Longitude'], return_counts=True)[1] / len(y_test)

# Print the original, training, and test class proportions for latitude
print("Original latitude class proportions:")
print(original_latitude_proportions)
print("\nTraining latitude class proportions:")
print(train_latitude_proportions)
print("\nTest latitude class proportions:")
print(test_latitude_proportions)

# Print the original, training, and test class proportions for longitude
print("\nOriginal longitude class proportions:")
print(original_longitude_proportions)
print("\nTraining longitude class proportions:")
print(train_longitude_proportions)
print("\nTest longitude class proportions:")
print(test_longitude_proportions)

Original latitude class proportions:
Latitude
-26.0521    0.16
-26.0501    0.15
-26.0365    0.12
-26.1076    0.12
-26.0722    0.12
-26.0557    0.09
-26.0305    0.06
-26.0698    0.06
-26.0874    0.06
-26.0412    0.06
Name: proportion, dtype: float64

Training latitude class proportions:
[0.12857143 0.05714286 0.12857143 0.05714286 0.08571429 0.15714286
 0.15714286 0.05714286 0.11428571 0.05714286]

Test latitude class proportions:
[0.1        0.06666667 0.1        0.06666667 0.1        0.16666667
 0.13333333 0.06666667 0.13333333 0.06666667]

Original longitude class proportions:
Longitude
28.0241    0.16
28.0569    0.15
28.0561    0.12
28.0567    0.12
28.0720    0.12
27.9892    0.09
28.0337    0.06
28.0551    0.06
28.0606    0.06
28.0724    0.06
Name: proportion, dtype: float64

Training longitude class proportions:
[0.08571429 0.15714286 0.05714286 0.05714286 0.11428571 0.12857143
 0.15714286 0.05714286 0.12857143 0.05714286]

Test longitude class proportions:
[0.1        0.16666667 0

In [66]:
import numpy as np
from sklearn.model_selection import KFold
import tensorflow as tf

# Set random seeds for reproducibility
tf.random.set_seed(2024)
np.random.seed(2024)

# Define the architecture of your neural network
ANN_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)  # Output layer with 2 neurons for latitude and longitude
])

# Compile the model
ANN_model.compile(optimizer='adam', loss='mean_squared_error')

# Define the number of folds for cross-validation
num_folds = 10

# Initialize lists to store the evaluation loss for each fold
losses = []

# Initialize the KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=2024)

# Perform 10-fold cross-validation
for train_index, test_index in kf.split(X_normalized):
    # Split the data into training and test sets for this fold
    X_train_fold, X_test_fold = X_normalized[train_index], X_normalized[test_index]
    y_train_fold, y_test_fold = y.values[train_index], y.values[test_index]
    
    # Train the model on this fold
    history = ANN_model.fit(X_train_fold, y_train_fold, epochs=50, validation_split=0.2, batch_size=32, verbose=0)
    
    # Evaluate the model on the test set for this fold
    loss =ANN_model.evaluate(X_test_fold, y_test_fold)
    
    # Append the loss to the list
    losses.append(loss)

# Calculate the mean loss across all folds
mean_loss = np.mean(losses)

print("Mean Test Loss across 10 folds:", mean_loss)

1/1 [==============================] - 0s 26ms/step - loss: 0.0865
Mean Test Loss across 10 folds: 7.536569605767727


In [67]:
# Predict on the test set
y_pred = ANN_model.predict(X_test)
print(y_pred)

1/1 [==============================] - 0s 40ms/step
[[-26.104132  28.10414 ]
 [-26.480549  28.385975]
 [-26.475426  28.5617  ]
 [-26.204103  28.33531 ]
 [-25.425133  27.320507]
 [-25.477156  27.269213]
 [-26.923815  28.986744]
 [-26.311134  28.148829]
 [-26.00533   27.91229 ]
 [-26.194912  28.16133 ]
 [-26.204103  28.33531 ]
 [-26.137518  28.013237]
 [-26.182884  28.285307]
 [-26.694641  28.660625]
 [-25.546305  27.524584]
 [-26.18564   28.161165]
 [-26.517668  28.80287 ]
 [-26.194433  28.2438  ]
 [-26.066858  28.084494]
 [-26.041086  28.158865]
 [-25.900366  27.70267 ]
 [-26.420994  28.259798]
 [-26.448267  28.480883]
 [-26.480549  28.385975]
 [-25.796738  27.721956]
 [-25.698305  27.611107]
 [-23.323946  24.97196 ]
 [-26.137518  28.013237]
 [-26.575708  28.40526 ]
 [-25.663383  27.572027]]


In [68]:
# If not, you may need to reshape it accordingly
# Assuming y_pred has shape (n_samples, 2)
# If not, adjust the column names accordingly
predictions_df = pd.DataFrame({'Latitude': y_pred[:, 0], 'Longitude': y_pred[:, 1]})

# Specify the file path where you want to save the Excel file
file_path = 'C:\\Users\\Aphiwe.Magaya\\Downloads\\predictions.xlsx'

# Write the DataFrame to an Excel file
predictions_df.to_excel(file_path, index=False)

print("Predictions have been saved to", file_path)

Predictions have been saved to C:\Users\Aphiwe.Magaya\Downloads\predictions.xlsx


In [69]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, root_mean_squared_error

# Predict on the test set
y_pred = ANN_model.predict(X_test)

# Calculate Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# Calculate Root Mean Squared Error (RMSE)
rmse = root_mean_squared_error(y_test, y_pred)
print("Root Mean Squared Error (RMSE):", rmse)

1/1 [==============================] - 0s 31ms/step
Mean Absolute Percentage Error (MAPE): 0.014872489219236418
Mean Absolute Error (MAE): 0.4024916073608399
Root Mean Squared Error (RMSE): 0.6504628898684969


In [70]:
# Save the model
#ANN_model.save("C:\\Users\\Aphiwe.Magaya\\OneDrive - Altron Group\\Documents\\PhePhaApp\\phepha_app\\assets\\ANN_model.keras")

# Convert the model to TensorFlow Lite format
#converter = tf.lite.TFLiteConverter.from_keras_model(ANN_model)
#tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
#with open('C:\\Users\\Aphiwe.Magaya\\OneDrive - Altron Group\\Documents\\PhePhaApp\\phepha_app\\assets\\ANN_model.tflite', 'wb') as f:
#    f.write(tflite_model)

In [76]:
import pickle
import gcsfs
import pickle

# Initialize a GCS file system
fs = gcsfs.GCSFileSystem()

# Serialize the model
model_bytes = pickle.dumps(ANN_model)

# Write the model bytes to a file in GCS
with fs.open('phepha-bucket_1/ml_models/model.pkl', 'wb') as f:
    f.write(model_bytes)

In [74]:
#import joblib

# Save the best model to the specified directory
#ANN_model.save("C:\\Users\\Aphiwe.Magaya\\OneDrive - Altron Group\\Documents\\PhePhaApp\\phepha_app\\assets\\ANN_model.keras")
#joblib.dump(ANN_model, 'C:\\Users\\Aphiwe.Magaya\\OneDrive - Altron Group\\Documents\\PhePhaApp\\phepha_app\\assets\\ANN_model.pbtxt')

In [72]:
#from google.cloud import firestore

In [73]:
# Define the Firestore collection to save data into
#collection_name = 'coordinates'
# Loop through each row in the DataFrame and write it to Firestore
#for idx, row in predictions_df.iterrows():     
    # Create a document reference    
#    doc_ref = db.collection(collection_name).document(f'record_{idx}') 
#   geo_point = firestore.GeoPoint(row['Latitude'], row['Longitude'])
#    # Write data to Firestore   
#    doc_ref.set({           
#    u'geopoint': geo_point,
#    u'crimeLevel': "mlCrime",
#    u'raduis':100  
#    })
#print("Data written to Firestore successfully!")